# Проект

Обучение модели классификации комментариев на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.Значением метрики качества F1 должно быть не меньше 0.75. Данные находятся в файле /datasets/toxic_comments.csv(источник Яндекс.Практикум). Столбец text в нём содержит текст комментария, а toxic — целевой признак с разметкой о токсичности

## Подготовка

In [1]:
!pip install transformers==4.11.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 3.3 MB 35.0 MB/s 
     |████████████████████████████████| 880 kB 43.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f037ea1eabb3dd0b97c83ab62dd4626d33410c9569b46c74da10d473493fb76d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install  --upgrade bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3765 sha256=7a226031d204b7a95537dd471f9093024ae929c6b7fdb22ed291799e0122c0cc
  Stored in directory: /root/.cache/pip/wheels/bb/31/1b/c05f362e347429b7436954d1a2280fe464731e8f569123a848
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6794 sha256=e8f8dcdf44ca427e01e9beaa6ca2f1f51abf15af72ac4502352d06816c504e4a
  Stored in directory: /root/.cache/pip/wheels/94/f1/b4/0b98b1e94775da6a0b1130e342d22af05cd269e1172c19f40f
Successfully built bert erlastic


In [3]:
import lightgbm as ltb
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
import tensorflow_hub as hub
import time
import torch
import transformers
from google.colab import drive
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from itertools import product
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import class_weight
from textblob import TextBlob, Word
from time import time
from tqdm import notebook
from transformers import BertTokenizer, TFBertModel

In [4]:
RND_STATE = 125
drive.mount('/content/drive')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Проект Викишоп/toxic_comments.csv')

In [6]:
data.query('toxic == 1')

,text,toxic
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,Hey... what is it..\n@ | talk .\nWhat is it......,1
16,"Bye! \n\nDon't look, come or think of comming ...",1
42,You are gay or antisemmitian? \n\nArchangel WH...,1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
...,...,...
159494,"""\n\n our previous conversation \n\nyou fuckin...",1
159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1
159541,Your absurd edits \n\nYour absurd edits on gre...,1
159546,"""\n\nHey listen don't you ever!!!! Delete my e...",1


In [7]:
data.describe(include='all')

,text,toxic
count,159571,159571.000000
unique,159571,NaN
top,Explanation\nWhy the edits made under my usern...,NaN
freq,1,NaN
mean,NaN,0.101679
std,NaN,0.302226
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,0.000000


Видно, что комментарии не повторяются. Нет пропусков

Найдем соотношение классов, при дисбалансе нужно это учитывать

In [9]:
class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(data['toxic']),
                                        y = data['toxic']                                                    
                                    )
class_weights = dict(zip(np.unique(data['toxic']), class_weights))
class_weights

{0: 0.5565938358935721, 1: 4.917442218798151}

Виден сильный дисбаланс классов, будем добавлять параметр class_weight

In [10]:
# избавимся от стоп-слов
stop_words = set(stopwords.words('english')) 
count_tf_idf = TfidfVectorizer(stop_words=stop_words) 

In [11]:
# Передадим POS-теги для различных частей речи
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV
                }
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()    

In [12]:
# лемматизируем
def text_blob(text):
    text = TextBlob(text)
    return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in text.words])  

# оставим только слова, приведя их в нижний регистр
def clear_text(text):
    return " ".join(re.sub(r'[^a-z\' ]', ' ', text).split())  

In [13]:
%%time
data['text'] = data['text'].apply(lambda x: text_blob(clear_text(x.lower())))

CPU times: user 22min 45s, sys: 1min 17s, total: 24min 3s
Wall time: 24min 2s


In [14]:
print(data.query('toxic == 1'))


                                                     text  toxic
6            cocksucker before you piss around on my work      1
12      hey what be it talk what be it an exclusive gr...      1
16      bye do n't look come or think of comming back ...      1
42      you be gay or antisemmitian archangel white ti...      1
43                  fuck your filthy mother in the as dry      1
...                                                   ...    ...
159494  our previous conversation you fuck shit eat li...      1
159514                   you be a mischievious pubic hair      1
159541  your absurd edits your absurd edits on great w...      1
159546  hey listen do n't you ever delete my edits eve...      1
159554  and i 'm go to keep post the stuff u delete un...      1

[16225 rows x 2 columns]


In [15]:
# Разобьем выборку на обучающую и тестовую
train, test = train_test_split(data, test_size=0.5, random_state=RND_STATE, shuffle=True)

In [16]:
# разделим на признаки и цели
target_train = train['toxic']
features_train = train['text']

target_test = test['toxic']
features_test = test['text']

In [17]:
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

## Обучение

In [31]:
# Создаем списки этапов пайплайна и соответствующих моделей
vectorizers = ['tfidfvectorizer']
classifiers = ['logisticregression', 'decisiontreeclassifier', 'lgbmclassifier']

In [32]:
# Создаём словарь с набором всех используемых компонентов пайплайнов
all_models = {
              'tfidfvectorizer': TfidfVectorizer(),            
              'logisticregression': LogisticRegression(class_weight=class_weights, solver='lbfgs', max_iter=1000),
              'decisiontreeclassifier': DecisionTreeClassifier(class_weight=class_weights, random_state=RND_STATE), 
              'lgbmclassifier': ltb.LGBMClassifier(class_weight=class_weights, random_state=RND_STATE)
             }

# Создаём словарь с наборами гиперпараметров всех компонентов
all_params = {
              'tfidfvectorizer':        {'tfidfvectorizer__ngram_range': [(1,1), (1,2)],
                                         'tfidfvectorizer__use_idf': (True, False),
                                         'tfidfvectorizer__norm': ('l1', 'l2')},             
              'logisticregression':     {'logisticregression__C': [0.1, 1, 10],
                                         'logisticregression__penalty': ("l2", "none")},
              'decisiontreeclassifier': {'decisiontreeclassifier__max_depth': [5, 10, 15],
                                         'decisiontreeclassifier__min_samples_leaf': [1, 2]
                                        },              
              'lgbmclassifier':         {'lgbmclassifier__n_estimators': [30, 300],
                                         'lgbmclassifier__min_samples_leaf': [1, 2]
                                        }
             }             

In [33]:
%%time
# product - метод, позволяющий в одноуровневом цикле создать все возможные сочетания нескольких параметров
# Создаем пустой датафрейм для сохранения результатов всех экспериментов и запускаем основной цикл, 
# на каждой итерации которого генерируем очередное сочетание всех этапов пайплайна, создаем словарь гиперпараметров для пайплайна, 
# создаем сам пайплайн, выполняем обучение и оценку качества с помощью кросс-валидации, записываем результаты обучения в лог.
df_all_results = pd.DataFrame()

for i, stages in enumerate(product(vectorizers, classifiers), 1):

    vectorizer, classifier = stages

    pipe = make_pipeline(all_models[vectorizer], all_models[classifier])
    
    current_params = {}
    current_params.update(all_params[vectorizer])
    current_params.update(all_params[classifier])
    print(current_params)
    
    model = GridSearchCV(estimator=pipe, param_grid=current_params,
                         scoring='f1', cv=3, n_jobs=-1)
    model.fit(features_train, target_train)
        
    df_current_results = pd.DataFrame(data={'pipeline': i,
                                            'vectorizer': vectorizer,  
                                            'classifier': classifier,
                                            'params': model.cv_results_['params'],
                                            'score': model.cv_results_['mean_test_score'],
                                            'fit_time': model.cv_results_['mean_fit_time'],
                                            'best_estimator_': model.best_estimator_ })
 
    df_all_results = df_all_results.append(df_current_results)

{'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)], 'tfidfvectorizer__use_idf': (True, False), 'tfidfvectorizer__norm': ('l1', 'l2'), 'logisticregression__C': [0.1, 1, 10], 'logisticregression__penalty': ('l2', 'none')}


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


{'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)], 'tfidfvectorizer__use_idf': (True, False), 'tfidfvectorizer__norm': ('l1', 'l2'), 'decisiontreeclassifier__max_depth': [5, 10, 15], 'decisiontreeclassifier__min_samples_leaf': [1, 2]}
{'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)], 'tfidfvectorizer__use_idf': (True, False), 'tfidfvectorizer__norm': ('l1', 'l2'), 'lgbmclassifier__n_estimators': [30, 300], 'lgbmclassifier__min_samples_leaf': [1, 2]}


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 16min 57s, sys: 1min 10s, total: 18min 7s
Wall time: 5h 16min 43s


In [34]:
df_all_results.sort_values(by='score', ascending=False, inplace=True)
df_all_results.head(4)

,pipeline,vectorizer,classifier,params,score,fit_time,best_estimator_
38,1,tfidfvectorizer,logisticregression,"{'logisticregression__C': 10, 'logisticregress...",0.775754,73.620438,"(TfidfVectorizer(ngram_range=(1, 2)), Logistic..."
34,1,tfidfvectorizer,logisticregression,"{'logisticregression__C': 10, 'logisticregress...",0.767775,14.417860,"(TfidfVectorizer(ngram_range=(1, 2)), Logistic..."
30,3,tfidfvectorizer,lgbmclassifier,"{'lgbmclassifier__min_samples_leaf': 2, 'lgbmc...",0.766060,523.826730,"(TfidfVectorizer(ngram_range=(1, 2)), LGBMClas..."
14,3,tfidfvectorizer,lgbmclassifier,"{'lgbmclassifier__min_samples_leaf': 1, 'lgbmc...",0.766060,524.824617,"(TfidfVectorizer(ngram_range=(1, 2)), LGBMClas..."


В этой таблице мы видим, что лучшая модель LogisticRegression с параметрами 'C'=10, 'penalty'='l2'
Написание кода с помощью pipeline выглядит лаконично, прозрачно, помогает "увидеть" работу.

## Выводы

Получилось, что из всех моделей лучший результат у Логистической регрессии с подобранными гиперпараметрами. Предсказание чуть хуже, чем у дерева решений, но на обучение ушло заметно меньше времени. Плюс, дерево решений после подбора гиперпараметров стало давать недопустимые результаты.

In [38]:
# Проверим лучшую модель на тестовой выборке/
model = LogisticRegression(class_weight=class_weights, solver='lbfgs', max_iter=1000, C=10, penalty='l2')
model.fit(tf_idf_train, target_train)
predicted = model.predict(tf_idf_test)
f1 = f1_score(target_test, predicted).round(4)
print(f1)

0.7562


Результат достаточно хорошо, но попробуем построить модель с эмбеддингом BERT

In [39]:
# путь для автоматического скачивания модели BERT
# если я правильно понимаю, теперь не нужно указывать относительные пути
!wget http://files.deeppavlov.ai/deeppavlov_data/bert/conversational_cased_L-12_H-768_A-12_pt.tar.gz
!tar -xvzf conversational_cased_L-12_H-768_A-12_pt.tar.gz

--2022-07-08 13:59:24--  http://files.deeppavlov.ai/deeppavlov_data/bert/conversational_cased_L-12_H-768_A-12_pt.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 178.63.27.41
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.deeppavlov.ai/deeppavlov_data/bert/conversational_cased_L-12_H-768_A-12_pt.tar.gz [following]
--2022-07-08 13:59:25--  https://files.deeppavlov.ai/deeppavlov_data/bert/conversational_cased_L-12_H-768_A-12_pt.tar.gz
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 403402567 (385M) [application/octet-stream]
Saving to: ‘conversational_cased_L-12_H-768_A-12_pt.tar.gz’

conversational_case 100%[===================>] 384.71M  25.5MB/s    in 14s     

2022-07-08 13:59:40 (26.6 MB/s) - ‘conversational_cased_L-12_H-768_A-12_pt.tar.gz’ saved [40340

In [82]:
# инициализируем токенизатор
tokenizer = transformers.BertTokenizer(
    vocab_file='/content/drive/My Drive/Colab Notebooks/Проект Викишоп/vocab.txt')

print(tokenizer)

PreTrainedTokenizer(name_or_path='', vocab_size=28995, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [83]:
# инициализируем конфигурацию BertConfig 
config = transformers.BertConfig.from_json_file('/content/drive/My Drive/Colab Notebooks/Проект Викишоп/bert_config.json')

In [84]:
# инициализируем саму модель класса BertModel. Передадим ей файл с предобученной моделью и конфигурацией
model = transformers.BertModel.from_pretrained('/content/drive/My Drive/Colab Notebooks/Проект Викишоп/pytorch_model.bin', config=config)

Some weights of the model checkpoint at /content/drive/My Drive/Colab Notebooks/Проект Викишоп/pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [85]:
# сократим на треть нашу выборку, облегчив расчеты
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Проект Викишоп/toxic_comments.csv')
data = data.sample(n=5000)

# Разобьем выборку на обучающую и тестовую
train, test = train_test_split(data, test_size=0.25, random_state=RND_STATE, shuffle=True, stratify=data['toxic'])

# разделим на признаки и цели
target_train2 = train['toxic']
features_train2 = train['text']

target_test2 = test['toxic']
features_test2 = test['text']

In [86]:
def get_embeddings(x_train):
    # преобразуем тексты в эмбеддинги
    tokenized = x_train.apply(
        lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=256))

    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)

    padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

    attention_mask = np.where(padded != 0, 1, 0)

    # сделаем цикл по батчам. Отображать прогресс будет функция notebook()
    batch_size = 50
    # сделаем пустой список для хранения эмбеддингов твитов
    embeddings = []
    for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
            # преобразуем данные
            batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
            # преобразуем маску
            attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
            
            # для ускорения вычисления функцией no_grad()
            with torch.no_grad():
                # получим эмбеддинги для батча
                batch_embeddings = model(batch, attention_mask=attention_mask_batch)
            
            # из полученного тензора извлечём нужные элементы и добавим в список всех эмбеддингов
            embeddings.append(batch_embeddings[0][:,0,:].numpy())
    return embeddings        

In [87]:
features = np.concatenate(get_embeddings(features_train2)) 

  0%|          | 0/75 [00:00<?, ?it/s]

In [88]:
print(features.shape, target_train2.shape, features_train2.shape)

(3750, 768) (3750,) (3750,)


In [89]:
# LogisticRegression
model_ = LogisticRegression(solver='lbfgs', max_iter=1000)
model_.fit(features, target_train2)

LogisticRegression(max_iter=1000)

In [90]:
test = np.concatenate(get_embeddings(features_test2)) 

  0%|          | 0/25 [00:00<?, ?it/s]

In [91]:
predicted = model_.predict(test)
f1_ = f1_score(target_test2, predicted).round(4)
print(f1_)

0.7225


Выводы:

В первой части проекта используется метод подготовки текста для привычных алгоритмов классификации,а именно:
- избавление от мусорных слов,
- приведение к нижнему регистру,
- лемматизация.
Лучшие результаты показала модель Логистической регресии.

Во второй части запустили логистическую регрессию, но данные обработаны предобученной моделью BERT. Учитывая, что во втором случае исследовалась только часть всего текста, а именно 5000 комментариев, разделенных на обучающую и тестовую выборки, результат получился неплохой - 0.7225